In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import pickle

# prevent tensorflow from using GPU. Otherwise, run out of memory
# https://stackoverflow.com/questions/44552585/prevent-tensorflow-from-accessing-the-gpu
# os.environ["CUDA_VISIBLE_DEVICES"]="-1"

import tensorflow_hub as hub
import tensorflow as tf

import nltk
import regex as re
import emoji as em

import itertools as it

elmo = hub.Module("https://tfhub.dev/google/elmo/2")

W0415 13:55:06.545597 140146709792576 __init__.py:56] Some hub symbols are not available because TensorFlow version is less than 1.14


In [9]:
# FUNCTIONS

# process tweet
def preprocess(tweet):
    #define hashtag pattern
    hashtag_define = re.compile ('#')
    #define mention pattern 
    mentions_define = re.compile('@[^\s]+')
    #define link pattern
    link_define = re.compile('https?://[^\s]+')
    # Haystack define
    haystack_define = re.compile('(RT)')
    # define long spaces
    extra_spaces_define = re.compile('\s{2,}')
    #remove hashtags
    tweet_refine = hashtag_define.sub('',tweet)
    # remove mentions
    tweet_refine = mentions_define.sub('',tweet_refine)
    # remove links
    tweet_refine = link_define.sub('',tweet_refine)
    # remove haystack
    tweet_refine = haystack_define.sub('',tweet_refine)
    #replace long spaces with one space
    tweet_refine = extra_spaces_define.sub(' ',tweet_refine)
    # convert emoticons into words
    tweet_refine = em.demojize(tweet_refine)
    return tweet_refine

# Create tokens with simple split on whitespace
def simple_token(s):
    return s.split()

# Get token length
def token_length(s):
    return len(s)

# Create "embedding lists" of equal size -- pad with empty characters, e.g. ""
# https://stackoverflow.com/questions/24066904/most-pythonic-way-to-extend-a-list-to-exactly-a-certain-length
def pad_list(some_list, target_len):
    return some_list[:target_len] + [""]*(target_len - len(some_list))

# Get the elmo embeddings
def elmo_tweet_embedder(tokens,len_list):
    
    tokens_input = tokens #load a tweet
    tokens_length = len_list # get length of tweet

    #create embedding
    embedding_tensor = elmo(inputs={"tokens":tokens_input,"sequence_len":tokens_length},
                            signature="tokens", as_dict=True)["word_emb"] # <-- passing in a list instead of [word]

    with tf.Session() as sess:
        sess.run(tf.global_variables_initializer())
        embedding = sess.run(embedding_tensor)
        return embedding

# combine all together
def make_embeddings(df, max_seq_len):
    m = max_seq_len
    
    # create a token column in df
    df['tokens'] = df['text'].apply(simple_token)
    
    # pad the tokens
    df['tokens'] = df.apply(lambda x: pad_list(x['tokens'],m),axis=1)
    
    # split data into smaller batches of size 100. http://bit.ly/2P4J8HJ
    text_batches = [df['tokens'][i:i+5000] for i in range(0,df.shape[0],5000)]
    len_lists = [[m] * len(x) for x in text_batches]

    # create list of sentiments (y values)
    y = df['sent'].to_numpy()
    
    # make embeddings
    embeddings = []
    for i in range(0,len(text_batches)):
        print("batch",i)
        elmo_train = elmo_tweet_embedder(text_batches[i].tolist(),len_lists[i])
        embeddings.append(elmo_train)
        
    embeddings = np.concatenate(embeddings, axis=0)
    
    return embeddings, y

In [6]:
path = '/home/tim/Documents/Sentiment/Data/dictionary_raw'
file = 'vocab_complete.txt'

p = os.path.join(path,file)
p
col_names = ['text']
df = pd.read_csv(p,sep="\n", header=None,names=col_names)
df['sent'] = df['text']
df.head()

,text,sent
0,<user>,<user>
1,.,.
2,:,:
3,rt,rt
4,",",","


In [7]:
len(df['text'])

977185

In [ ]:
X, y = make_embeddings(df, 1)
d = dict()
for i in range(len(y)):
    d[y[i]] = X[i][0]

batch 0
batch 1
batch 2
batch 3
batch 4
batch 5
batch 6
batch 7
batch 8
batch 9
batch 10
batch 11
batch 12
batch 13
batch 14
batch 15
batch 16


In [29]:
df2 = df.iloc[0:5]
df2

,text,sent
0,<user>,<user>
1,.,.
2,:,:
3,rt,rt
4,",",","


In [30]:
X, y = make_embeddings(df2, 1)

/home/tim/anaconda3/envs/tf-gpu/lib/python3.6/site-packages/ipykernel_launcher.py:62: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/home/tim/anaconda3/envs/tf-gpu/lib/python3.6/site-packages/ipykernel_launcher.py:65: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [32]:
y

array(['<user>', '.', ':', 'rt', ','], dtype=object)

In [56]:
len(X[1][0])

512

In [37]:
d = dict(zip(X,y))

TypeError: unhashable type: 'numpy.ndarray'

In [61]:
d = dict()
for i in range(len(y)):
    d[y[i]] = X[i][0]

In [62]:
d

{'<user>': array([-1.28953338e+00,  2.64393657e-01,  2.83248186e-01, -3.33554626e-01,
         1.91184312e-01, -1.27828300e+00,  1.73725322e-01, -3.24325025e-01,
        -1.03575826e+00,  1.59545213e-01,  2.46927142e-01,  1.15280665e-01,
         4.98969167e-01,  5.75006247e-01, -2.44155511e-01,  1.71055615e-01,
         2.99325407e-01, -4.02063392e-02,  6.14031076e-01,  1.04519099e-01,
         6.33725941e-01,  4.90864843e-01,  3.33329380e-01, -3.88322234e-01,
         3.87545139e-01, -1.34653777e-01,  1.45469353e-01,  5.55816650e-01,
        -4.02791530e-01, -5.37024438e-02, -6.39737368e-01,  6.77715018e-02,
        -8.78484845e-02, -4.39286977e-02, -9.83125269e-02, -8.25847626e-01,
        -1.93029165e-01, -1.86266974e-01,  7.64780283e-01, -5.85160673e-01,
        -1.79870576e-01, -6.18967451e-02,  2.23275185e-01,  3.24645638e-03,
         4.46756870e-01, -1.94461301e-01, -8.84806633e-01, -8.03327620e-01,
        -7.78932810e-01, -5.87615132e-01,  1.12967692e-01,  1.45000517e-01,
  

In [9]:
path = '/home/tim/Documents/Sentiment/Data/processed/'
file_list = os.listdir(path)

# create the data set names
names = []
for f in file_list:
    f = f.split('.')
    l = ["x_"+f[0],"y_"+f[0]]
    names.append(l)

data_names = names
    
# index through files and create x and y sets
for i in range(0,len(file_list)):
    
    s = names
   
    p = os.path.join(path,file_list[i])
    col_names = ['text','sent']
    df = pd.read_csv(p,delimiter="\t",names=col_names)

    print(s[i][0], "start")
    s[i][0], s[i][1] = make_embeddings(df, 80) 

x_train = s[0][0]
y_train = s[0][1]
x_dev = s[1][0]
y_dev = s[1][1]
x_test = s[2][0]
y_test = s[2][1]

x_train start
x_dev start
x_test start


In [15]:
# Create Pickles
pickle_path = '/home/tim/Documents/Sentiment/Data/pickles'

# x_train
pickle_out = open(os.path.join(pickle_path,'x_train.pickle'),"wb")
pickle.dump(x_train, pickle_out)
pickle_out.close()

# y_train
pickle_out = open(os.path.join(pickle_path,'y_train.pickle'),"wb")
pickle.dump(y_train, pickle_out)
pickle_out.close()

# x_dev
pickle_out = open(os.path.join(pickle_path,'x_dev.pickle'),"wb")
pickle.dump(x_dev, pickle_out)
pickle_out.close()

# y_dev
pickle_out = open(os.path.join(pickle_path,'y_dev.pickle'),"wb")
pickle.dump(y_dev, pickle_out)
pickle_out.close()

# x_test
pickle_out = open(os.path.join(pickle_path,'x_test.pickle'),"wb")
pickle.dump(x_test, pickle_out)
pickle_out.close()

# y_test
pickle_out = open(os.path.join(pickle_path,'y_test.pickle'),"wb")
pickle.dump(y_test, pickle_out)
pickle_out.close()